In [ ]:
import os
import pandas as pd
import ast

from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.core import Composition
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility
import re

Preparation

In [7]:
##Import MACE raw data
df_mace_normal_oxide = pd.read_csv('../../data/MACE/results_oxide_normal_all.csv')
df_mace_normal_oxide.rename(columns={'Compositions': 'Formula'},inplace=True)
df_mace_inverse_oxide = pd.read_csv('../../data/MACE/results_oxide_inverse_all.csv')
df_mace_inverse_oxide.rename(columns={'Compositions': 'Formula'},inplace=True)
df_mace_normal_sulfide = pd.read_csv('../../data/MACE/results_sulfide_normal_all.csv')
df_mace_normal_sulfide.rename(columns={'Compositions': 'Formula'},inplace=True)
df_mace_inverse_sulfide = pd.read_csv('../../data/MACE/results_sulfide_inverse_all.csv')
df_mace_inverse_sulfide.rename(columns={'Compositions': 'Formula'},inplace=True)
df_mace_normal_selenide = pd.read_csv('../../data/MACE/results_selenide_normal_all.csv')
df_mace_normal_selenide.rename(columns={'Compositions': 'Formula'},inplace=True)
df_mace_inverse_selenide = pd.read_csv('../../data/MACE/results_selenide_inverse_all.csv')
df_mace_inverse_selenide.rename(columns={'Compositions': 'Formula'},inplace=True)
df_mace_normal_telluride = pd.read_csv('../../data/MACE/results_telluride_normal_all.csv')
df_mace_normal_telluride.rename(columns={'Compositions': 'Formula'},inplace=True)
df_mace_inverse_telluride = pd.read_csv('../../data/MACE/results_telluride_inverse_all.csv')
df_mace_inverse_telluride.rename(columns={'Compositions': 'Formula'},inplace=True)

In [4]:
##Combine normal and inverse dataframes and drop unused columns
df_mace_oxide_all = pd.merge(df_mace_normal_oxide.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), df_mace_inverse_oxide.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), on='Formula', how='outer', suffixes=('_normal', '_inverse'))
df_mace_sulfide_all = pd.merge(df_mace_normal_sulfide.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), df_mace_inverse_sulfide.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), on='Formula', how='outer', suffixes=('_normal', '_inverse'))
df_mace_selenide_all = pd.merge(df_mace_normal_selenide.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), df_mace_inverse_selenide.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), on='Formula', how='outer', suffixes=('_normal', '_inverse'))
df_mace_telluride_all = pd.merge(df_mace_normal_telluride.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), df_mace_inverse_telluride.drop(columns=['a','b','c','angle_a','angle_b','angle_c','volume','force']), on='Formula', how='outer', suffixes=('_normal', '_inverse'))

In [5]:
df_mace_oxide_all

,Formula,energy_normal,steps_normal,conv_normal,force_conv_normal,energy_inverse,steps_inverse,conv_inverse,force_conv_inverse
0,AgAg2O4,-219.903159,70,pass,pass,-219.903159,70,pass,pass
1,AgAl2O4,-358.219855,37,pass,pass,-353.549602,72,pass,pass
2,AgAs2O4,-294.125390,65,pass,pass,-305.021529,378,pass,pass
3,AgAu2O4,-229.236827,79,pass,pass,-229.367311,273,pass,pass
4,AgB2O4,-316.575469,53,pass,pass,-378.372131,447,pass,pass
...,...,...,...,...,...,...,...,...,...
5052,ZrW2O4,-441.718972,77,pass,pass,-441.281560,248,pass,pass
5053,ZrY2O4,-499.782902,83,pass,pass,-503.675540,157,pass,pass
5054,ZrYb2O4,-433.400235,75,pass,pass,-433.508404,108,pass,pass
5055,ZrZn2O4,-363.621126,47,pass,pass,-366.942533,100,pass,pass


In [6]:
##Find MACE converged structures
def find_mace_converged(df):
    df['MACE_conv_normal'] = df['Formula'].isin(df[(df['steps_normal']<600) & (df['conv_normal']=='pass') & (df['force_conv_normal']=='pass')]['Formula'])
    df['MACE_conv_inverse'] = df['Formula'].isin(df[(df['steps_inverse']<600) & (df['conv_inverse']=='pass') & (df['force_conv_inverse']=='pass')]['Formula'])
    df['MACE_conv_both'] = df['Formula'].isin(df[(df['MACE_conv_normal']==True) & (df['MACE_conv_inverse']==True)]['Formula'])
    df['MACE_conv_any'] = df['Formula'].isin(df[(df['MACE_conv_normal']==True) | (df['MACE_conv_inverse']==True)]['Formula'])
    return df

df_mace_oxide_all = find_mace_converged(df_mace_oxide_all)
df_mace_sulfide_all = find_mace_converged(df_mace_sulfide_all)
df_mace_selenide_all = find_mace_converged(df_mace_selenide_all)
df_mace_telluride_all = find_mace_converged(df_mace_telluride_all)

In [ ]:
##Prepare data for Ehull calculation, separate data into structures that converged in normal and inverse configurations
df_mace_oxide_energy_converged_normal = df_mace_oxide_all[df_mace_oxide_all['MACE_conv_normal']==True][['Formula','energy_normal']]
df_mace_oxide_energy_converged_inverse = df_mace_oxide_all[df_mace_oxide_all['MACE_conv_inverse']==True][['Formula','energy_inverse']]
df_mace_sulfide_energy_converged_normal = df_mace_sulfide_all[df_mace_sulfide_all['MACE_conv_normal']==True][['Formula','energy_normal']]
df_mace_sulfide_energy_converged_inverse = df_mace_sulfide_all[df_mace_sulfide_all['MACE_conv_inverse']==True][['Formula','energy_inverse']]
df_mace_selenide_energy_converged_normal = df_mace_selenide_all[df_mace_selenide_all['MACE_conv_normal']==True][['Formula','energy_normal']]
df_mace_selenide_energy_converged_inverse = df_mace_selenide_all[df_mace_selenide_all['MACE_conv_inverse']==True][['Formula','energy_inverse']]
df_mace_telluride_energy_converged_normal = df_mace_telluride_all[df_mace_telluride_all['MACE_conv_normal']==True][['Formula','energy_normal']]
df_mace_telluride_energy_converged_inverse = df_mace_telluride_all[df_mace_telluride_all['MACE_conv_inverse']==True][['Formula','energy_inverse']]

##Calculate energy per atom, in this case, all spinels have 56 atoms per formula unit
df_mace_oxide_energy_converged_normal['EnergyPerAtom'] = df_mace_oxide_energy_converged_normal['energy_normal'] / 56
df_mace_oxide_energy_converged_inverse['EnergyPerAtom'] = df_mace_oxide_energy_converged_inverse['energy_inverse'] / 56
df_mace_sulfide_energy_converged_normal['EnergyPerAtom'] = df_mace_sulfide_energy_converged_normal['energy_normal'] / 56
df_mace_sulfide_energy_converged_inverse['EnergyPerAtom'] = df_mace_sulfide_energy_converged_inverse['energy_inverse'] / 56
df_mace_selenide_energy_converged_normal['EnergyPerAtom'] = df_mace_selenide_energy_converged_normal['energy_normal'] / 56
df_mace_selenide_energy_converged_inverse['EnergyPerAtom'] = df_mace_selenide_energy_converged_inverse['energy_inverse'] / 56
df_mace_telluride_energy_converged_normal['EnergyPerAtom'] = df_mace_telluride_energy_converged_normal['energy_normal'] / 56
df_mace_telluride_energy_converged_inverse['EnergyPerAtom'] = df_mace_telluride_energy_converged_inverse['energy_inverse'] / 56

##Extract elements from formula for Ehull calculation
df_mace_oxide_energy_converged_normal['Elements'] = df_mace_oxide_energy_converged_normal['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))
df_mace_oxide_energy_converged_inverse['Elements'] = df_mace_oxide_energy_converged_inverse['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))
df_mace_sulfide_energy_converged_normal['Elements'] = df_mace_sulfide_energy_converged_normal['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))
df_mace_sulfide_energy_converged_inverse['Elements'] = df_mace_sulfide_energy_converged_inverse['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))
df_mace_selenide_energy_converged_normal['Elements'] = df_mace_selenide_energy_converged_normal['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))
df_mace_selenide_energy_converged_inverse['Elements'] = df_mace_selenide_energy_converged_inverse['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))
df_mace_telluride_energy_converged_normal['Elements'] = df_mace_telluride_energy_converged_normal['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))
df_mace_telluride_energy_converged_inverse['Elements'] = df_mace_telluride_energy_converged_inverse['Formula'].apply(lambda x: re.findall(r'[A-Z][a-z]?', x))

In [9]:
df_mace_oxide_energy_converged_normal

,Formula,energy_normal,EnergyPerAtom,Elements
0,AgAg2O4,-219.903159,-3.926842,"[Ag, Ag, O]"
1,AgAl2O4,-358.219855,-6.396783,"[Ag, Al, O]"
2,AgAs2O4,-294.125390,-5.252239,"[Ag, As, O]"
3,AgAu2O4,-229.236827,-4.093515,"[Ag, Au, O]"
4,AgB2O4,-316.575469,-5.653133,"[Ag, B, O]"
...,...,...,...,...
5052,ZrW2O4,-441.718972,-7.887839,"[Zr, W, O]"
5053,ZrY2O4,-499.782902,-8.924695,"[Zr, Y, O]"
5054,ZrYb2O4,-433.400235,-7.739290,"[Zr, Yb, O]"
5055,ZrZn2O4,-363.621126,-6.493234,"[Zr, Zn, O]"


Ehull calculations

In [ ]:
##Function to calculate Ehull, results are saved in a csv file
def ehull(df, range_to_run, config, anion):
    #set output file
    results_columns = ['Formula', 'Ehull']
    output_csv = f'calculation_scripts/Ehull/list_converged_{config}_{anion}_ehull.csv'

    # Write the header to the CSV file initially (only once)
    df_header = pd.DataFrame(columns=results_columns)
    df_header.to_csv(output_csv, index=False, header=True)

    for i in range_to_run:
        try:    
            with MPRester("G76W9BmY62dEoc8zYnQBd9KvZCWCZUQf") as mpr:
                entries = mpr.get_entries_in_chemsys(df.iloc[i]['Elements'])
                # Use uncorrected entries to generate the phase diagram
                uncorrected_entries = [
                    ComputedEntry(composition=e.composition, energy=e.uncorrected_energy)
                    for e in entries
                ]
            comp = Composition(df.iloc[i]['Formula'])
            mace_energy = df.iloc[i]['EnergyPerAtom']*7  # use mace energy in eV, since there are 7 atoms in the formula unit we multiply energy per atom by 7 here
            my_entry = ComputedEntry(comp, mace_energy)

            # Suppose you have multiple entries
            # my_entries = [my_entry, my_entry]

            # Combine your new entry with the existing entries from the MP database
            total_entries = uncorrected_entries + [my_entry]

            # Build the phase diagram
            phase_diag = PhaseDiagram(total_entries)

            # Get the energy above hull for your entry
            E_above_hull = phase_diag.get_e_above_hull(my_entry)
            df_result = pd.DataFrame([(df.iloc[i]['Formula'], E_above_hull)])
        except:
            df_result = pd.DataFrame([(df.iloc[i]['Formula'], 'error')])

        # Append the new row to the CSV file
        with open(output_csv, 'a') as f:
            df_result.to_csv(f, index=False, header=False)
    return None

In [ ]:
## Select configuration and anion
config = 'normal'
## config = 'inverse'

anion = 'oxide'
# anion = 'sulfide'
# anion = 'selenide'
# anion = 'telluride'

## Select dataframe to run
calculation_to_run = df_mace_oxide_energy_converged_normal
# calculation_to_run = df_mace_oxide_energy_converged_inverse
# calculation_to_run = df_mace_sulfide_energy_converged_normal
# calculation_to_run = df_mace_sulfide_energy_converged_inverse
# calculation_to_run = df_mace_selenide_energy_converged_normal
# calculation_to_run = df_mace_selenide_energy_converged_inverse
# calculation_to_run = df_mace_telluride_energy_converged_normal
# calculation_to_run = df_mace_telluride_energy_converged_inverse

##Running calculations
range_to_run = range(0, len(calculation_to_run)) #change this if want to perform test run on smaller number
ehull(calculation_to_run, range_to_run, config, anion)